In [67]:
import re
from urllib.parse import unquote, urlparse
from collections import defaultdict
import pandas as pd

# Load the data from CSV
df = pd.read_csv('./input/heimlicher.com_access.csv')
# Convert 'Last' to a datetime object
df['Last'] = pd.to_datetime(df['Last'], format='%d/%b/%Y:%H:%M:%S %z', utc=True)

In [68]:
# Consider only URLs that might be relavant

# Ignore URLs that contain malicious code
malicous_regex = r"^/data|^/api|&amp|\\x22|select*|/RK=0|/RS=\^|'x'|\+\+\+\+|2wCEAAgGBgcGB|vWfM6kbCUIv|fa3c615d773|iVBORw0KGgo"
malicious_mask = df['URI'].str.contains(malicous_regex, na=False, case=False, regex=True)

# Ignore URLs that contain URL-encoded characters such as %20|%23|%C3%(?:83|AE|AF|A2|82|html)|%E6%88|%22%20class=%22|...
encoded_mask_to_ignore = df['URI'].str.contains(r'%[0-9A-F]{2}', na=False, case=False, regex=True)
# ...but keep URLs that contain `Page%28[^%]+%29` as those occur in URLs of the form `Page(/articles/_index.md)`
encoded_mask_to_keep = df['URI'].str.contains(r'Page%28[^%]+index\.md%29', na=False, case=False, regex=True)
encoded_mask = encoded_mask_to_ignore & ~encoded_mask_to_keep

# Ignore URLs that contain 'http:' or 'https:'...
http_mask_to_ignore = df['URI'].str.contains(r'https?:', na=False, case=False, regex=True)
# ...but keep URLs that contain 'heimlicher.com'
http_heimlicher_mask_to_keep = df['URI'].str.contains(r'https?://(?:simon)\.heimlicher\.com', na=False, case=False, regex=True)
http_mask = http_mask_to_ignore & ~http_heimlicher_mask_to_keep

# Ignore URLs that contain '.php'...
php_mask_to_ignore = df['URI'].str.contains('.php', na=False, case=False, regex=False)
# ... unless they contain 'doku.php'
doku_php_mask_to_keep = df['URI'].str.contains('/doku.php', na=False, case=False, regex=False)
php_mask = php_mask_to_ignore & ~doku_php_mask_to_keep

# Ignore URLs that contain a file extension...
file_extension_mask_to_ignore = df['URI'].str.contains(r'.', na=False, case=False, regex=False)
# ...unless they ontain '.pdf' or '.md'
file_extension_mask_to_keep = df['URI'].str.contains(r'\.(?:xml|html|pdf|md)', na=False, case=False, regex=True)
file_extension_mask = file_extension_mask_to_ignore & ~file_extension_mask_to_keep

ignore_mask = malicious_mask | encoded_mask | http_mask | php_mask | file_extension_mask

# Split the DataFrame into two parts
df_valid = df[~ignore_mask]
df_valid = df_valid.groupby('URI').agg({
    'Count': 'sum',         # Sum the 'Count'
    'Last': 'max'           # Get the most recent 'Last' date
}).reset_index()
df_valid.sort_values(['Count', 'Last'], ascending=False, inplace=True)
df_valid

,URI,Count,Last
2,/,1939518,2024-01-07 12:32:07+00:00
1323,/about,185491,2024-01-03 02:34:06+00:00
2959,/atom.xml,170375,2024-01-07 09:54:37+00:00
2215,/articles/2011/07/26/disable-recent-items,148946,2024-01-06 18:32:38+00:00
2392,/articles/2012/07/10/time-machine-inherit-back...,104359,2024-01-07 12:21:01+00:00
...,...,...,...
1958,/articles/2009/,1,2013-12-16 19:45:29+00:00
1997,/articles/2009/11/,1,2013-12-16 19:45:29+00:00
2160,/articles/2011/03/17/cisco-vpn-10.6.0-3.html,1,2013-12-12 16:06:00+00:00
1970,/articles/2009/01/10/time-machine-readonly.html,1,2013-12-12 16:05:57+00:00


In [69]:
# Add a column that contains the URL without query
def sanitize_url(url):
    # Replace multiple slashes with a single slash
    url = re.sub(r'(?<!:)//+', '/', url)
    return url

def url_without_query(url):
    # First, sanitize the URL
    sanitized_url = sanitize_url(url)

    # Parse the sanitized URL
    parsed_url = urlparse(sanitized_url)
    
    # Check if the URL is a complete URL (includes a scheme like http)
    if parsed_url.scheme and parsed_url.netloc:
        return parsed_url.scheme + "://" + parsed_url.netloc + parsed_url.path
    elif parsed_url.path.startswith('/'):
        # If it's just a path starting with '/', return the path
        return parsed_url.path
    else:
        # Invalid URL
        return None

# Apply the function to create a new column
df_valid['URL Without Query'] = df_valid['URI'].apply(url_without_query)

# Create a mask to keep only non-None entries in 'URL Without Query'
valid_url_mask = df_valid['URL Without Query'].notnull()

# Apply the mask to filter the DataFrame
df_valid_filtered = df_valid[valid_url_mask]

# Group by 'URL Without Query' and sum 'Count'
df_cleaned = df_valid_filtered.groupby('URL Without Query').agg({
    'Count': 'sum',    # Sum the 'Count'
    'Last': 'max'            # Get the most recent 'Last' date
}).reset_index()

# Sort by 'URL Without Query'
df_cleaned.rename(columns={'URL Without Query': 'URI'}, inplace=True)
df_cleaned.sort_values(by=['Count', 'Last'], ascending=False, inplace=True)
df_cleaned.to_csv('./output/accesslog_urls.csv', index=False)
df_cleaned

,URI,Count,Last
0,/,1944694,2024-01-07 12:32:07+00:00
653,/about,185537,2024-01-06 16:26:33+00:00
2210,/atom.xml,178818,2024-01-07 09:54:37+00:00
1518,/articles/2011/07/26/disable-recent-items,148956,2024-01-06 18:32:38+00:00
1682,/articles/2012/07/10/time-machine-inherit-back...,104450,2024-01-07 12:21:01+00:00
...,...,...,...
1273,/articles/2009/,1,2013-12-16 19:45:29+00:00
1309,/articles/2009/11/,1,2013-12-16 19:45:29+00:00
1465,/articles/2011/03/17/cisco-vpn-10.6.0-3.html,1,2013-12-12 16:06:00+00:00
1285,/articles/2009/01/10/time-machine-readonly.html,1,2013-12-12 16:05:57+00:00


In [70]:

# Add a column that contains a canonicalized form of the URL
canonicalization_rules = [
    (r'^/Page(?:[(]|%28)(.*)(?:(/[^./]*)|(?:/_?index\.md))(?:[)]|%29)$', r'^/Page(?:[(]|%28)(.*)(?:(/[^./]*)|(?:/_?index\.md))(?:[)]|%29)$', r'\1\2/'),
    (r'^/articles/[0-9]{4}', r'^(/articles/)(?:[0-9]{4}/[0-9]{2}/[0-9]{2}/)([^/]+).*$', r'\1\2/'),
    (r'.*/(?:atom|index)', r'(.*?/)(?:atom|index).*', r'\1'),
    (r'.*/page/[1-9]*/?$', r'(.*?/)page/[1-9].*', r'\1'),
    (r'^/hints', r'_', r'-'),
    (r'.*\.html', r'(.*?)(?:/index)?\.html.*', r'\1/'),
    (r'^(.*/[^./]+)$', r'(^.*/[^./]+)$', r'\1/'),
]

def canonicalize_url(row):    
    url = row['URI']
    canonical_url = url
    # Apply all canonicalization rules in sequence
    for pattern, search_pattern, replacement in canonicalization_rules:
        if re.match(pattern, canonical_url):
            canonical_url = re.sub(search_pattern, replacement, canonical_url)
            
    return canonical_url

# Apply the function and create 'df_redirects_raw'
df_canonicalized = df_cleaned.copy()
df_canonicalized['Canonical'] = df_cleaned.apply(canonicalize_url, axis=1)
df_canonicalized = df_canonicalized[['URI', 'Canonical', 'Count', 'Last']].reset_index()
df_canonicalized

,index,URI,Canonical,Count,Last
0,0,/,/,1944694,2024-01-07 12:32:07+00:00
1,653,/about,/about/,185537,2024-01-06 16:26:33+00:00
2,2210,/atom.xml,/,178818,2024-01-07 09:54:37+00:00
3,1518,/articles/2011/07/26/disable-recent-items,/articles/disable-recent-items/,148956,2024-01-06 18:32:38+00:00
4,1682,/articles/2012/07/10/time-machine-inherit-back...,/articles/time-machine-inherit-backup-using-tm...,104450,2024-01-07 12:21:01+00:00
...,...,...,...,...,...
7275,1273,/articles/2009/,/articles/2009/,1,2013-12-16 19:45:29+00:00
7276,1309,/articles/2009/11/,/articles/2009/11/,1,2013-12-16 19:45:29+00:00
7277,1465,/articles/2011/03/17/cisco-vpn-10.6.0-3.html,/articles/cisco-vpn-10.6.0-3/,1,2013-12-12 16:06:00+00:00
7278,1285,/articles/2009/01/10/time-machine-readonly.html,/articles/time-machine-readonly/,1,2013-12-12 16:05:57+00:00


In [71]:
transformation_rules = [
    (r'^/publications.*', r'^(/publications/.*)', r'/research\1'),
    (r'^/(?:publications/)?(dissertation|characterizing-networks|forwarding-paradigms)', r'^/(?:publications/)?(dissertation|characterizing-networks|forwarding-paradigms)', r'/research\1'),
    # (r'^/hints', r'^/hints(?:/[^./]+)*/([^./]+)/*$', r'/articles/\1/'),
]

def transform_url(row):
    url = row['Canonical']

    # Apply transformation rules
    for pattern, search_pattern, replacement in transformation_rules:
        if re.match(pattern, url):
            url = re.sub(search_pattern, replacement, url)

    return url

# Apply the function and create 'df_redirects_raw'
df_redirects_raw = df_canonicalized.copy()
df_redirects_raw['Redirect URI'] = df_canonicalized.apply(transform_url, axis=1)

# Filter to create 'df_redirects'
df_redirects = df_redirects_raw[df_redirects_raw['Redirect URI'].notnull()]
df_redirects = df_redirects[['URI', 'Canonical', 'Redirect URI', 'Count', 'Last']].reset_index()

df_redirects.to_csv('./output/accesslog_redirects.csv', index=False)
df_redirects

,index,URI,Canonical,Redirect URI,Count,Last
0,0,/,/,/,1944694,2024-01-07 12:32:07+00:00
1,1,/about,/about/,/about/,185537,2024-01-06 16:26:33+00:00
2,2,/atom.xml,/,/,178818,2024-01-07 09:54:37+00:00
3,3,/articles/2011/07/26/disable-recent-items,/articles/disable-recent-items/,/articles/disable-recent-items/,148956,2024-01-06 18:32:38+00:00
4,4,/articles/2012/07/10/time-machine-inherit-back...,/articles/time-machine-inherit-backup-using-tm...,/articles/time-machine-inherit-backup-using-tm...,104450,2024-01-07 12:21:01+00:00
...,...,...,...,...,...,...
7275,7275,/articles/2009/,/articles/2009/,/articles/2009/,1,2013-12-16 19:45:29+00:00
7276,7276,/articles/2009/11/,/articles/2009/11/,/articles/2009/11/,1,2013-12-16 19:45:29+00:00
7277,7277,/articles/2011/03/17/cisco-vpn-10.6.0-3.html,/articles/cisco-vpn-10.6.0-3/,/articles/cisco-vpn-10.6.0-3/,1,2013-12-12 16:06:00+00:00
7278,7278,/articles/2009/01/10/time-machine-readonly.html,/articles/time-machine-readonly/,/articles/time-machine-readonly/,1,2013-12-12 16:05:57+00:00


In [72]:
df_redirects_2018_10 = df_redirects[
    (df_redirects['Last'].dt.year >= 2018) &             # Last Access is in the year 2018 or later
    (df_redirects['Count'] > 10)                  # Total Access Count is greater than 10
]

df_redirects_2018_10.to_csv('./output/accesslog_redirects_after2018_min10.csv', index=False)
df_redirects_2018_10

,index,URI,Canonical,Redirect URI,Count,Last
0,0,/,/,/,1944694,2024-01-07 12:32:07+00:00
1,1,/about,/about/,/about/,185537,2024-01-06 16:26:33+00:00
2,2,/atom.xml,/,/,178818,2024-01-07 09:54:37+00:00
3,3,/articles/2011/07/26/disable-recent-items,/articles/disable-recent-items/,/articles/disable-recent-items/,148956,2024-01-06 18:32:38+00:00
4,4,/articles/2012/07/10/time-machine-inherit-back...,/articles/time-machine-inherit-backup-using-tm...,/articles/time-machine-inherit-backup-using-tm...,104450,2024-01-07 12:21:01+00:00
...,...,...,...,...,...,...
1668,1668,/WebAdmin/FCKeditor/editor/fckeditor.html,/WebAdmin/FCKeditor/editor/fckeditor/,/WebAdmin/FCKeditor/editor/fckeditor/,11,2018-07-22 14:19:18+00:00
1669,1669,/_engine/fckeditor/editor/fckeditor.html,/_engine/fckeditor/editor/fckeditor/,/_engine/fckeditor/editor/fckeditor/,11,2018-07-22 14:19:18+00:00
1670,1670,/register/,/register/,/register/,11,2018-07-22 14:02:04+00:00
1671,1671,/admin/basic,/admin/basic/,/admin/basic/,11,2018-07-22 14:01:39+00:00


In [73]:
# File path of your list
urls_path = '../../output/serve/devel/_urls'

processed_url_lines = []
with open(urls_path, 'r') as file:
    for line in file:
        # Split the line at the first unquoted '#'
        parts = line.split('#', 1)
        cleaned_line = parts[0].strip()  # Keep only the part before the '#'
        
        # Skip empty lines
        if cleaned_line:
            processed_url_lines.append(cleaned_line)

# Convert the processed lines into a DataFrame
df_valid_urls = pd.DataFrame(processed_url_lines, columns=['URI'])
df_valid_urls

,URI
0,/
1,/401.html
2,/403.html
3,/404.html
4,/500.html
...,...
96,/de/technology/vscode-quick-open-recently/
97,/de/transformation/
98,/de/transformation/how-objectives-key-results-...
99,/de/transformation/okrs-vs-performance-managem...


In [74]:
aliases_path = '../../output/serve/devel/_aliases'

processed_alias_lines = []
with open(aliases_path, 'r') as file:
    for line in file:
        # Split the line at the first unquoted '#'
        parts = line.split('#', 1)
        cleaned_line = parts[0].strip()  # Keep only the part before the '#'
        
        # Skip empty lines
        if cleaned_line:
            processed_alias_lines.append(cleaned_line)

# Convert the processed lines into a DataFrame
df_alias_redirects = pd.DataFrame([line.split() for line in processed_alias_lines], columns=['URI', 'Redirect URI', 'Redirect status'])
df_alias_redirects

,URI,Redirect URI,Redirect status
0,/archive/,/,301
1,/articles/,/leadership/,301
2,/articles/acm-template-with-basic-tex-live/,/technology/,301
3,/articles/capitalization-title-headings-prefer...,/digitization/capitalization-title-headings-pr...,301
4,/articles/capitalization/,/digitization/capitalization-title-headings-pr...,301
...,...,...,...
82,/tags/vpn/,/tags/macos/,301
83,/tags/windows/,/tags/,301
84,/tags/writing/,/tags/communication/,301
85,/transformation/organizational-transformation-...,/transformation/how-objectives-key-results-fac...,301


In [75]:
df_merged_redirects = pd.DataFrame(columns=['URI', 'Canonical', 'Redirect URI', 'Redirect status', 'Count', 'Last'])
merged_redirects_list = []

REDIRECT_STATUS_OK = 200
REDIRECT_STATUS_FOUND = 301
REDIRECT_STATUS_NOT_FOUND = 404

for index, row in df_redirects.iterrows():
    
    # Default to no redirect, i.e., column 'Redirect URI' = column 'URI' and status = 200
    new_row = {
        'URI': row['URI'],
        'Canonical': row['Canonical'],
        'Redirect URI': row['URI'],
        'Redirect status': REDIRECT_STATUS_OK,
        'Count': row['Count'],
        'Last': row['Last'],
    }

    # 1. Determine if the URL we try to redirect is actually valid and should not be redirected at all
    url_match = df_valid_urls[df_valid_urls['URI'] == row['URI']]
    if url_match.empty:
        
        #
        # The URL in the 'URI' column is not a valid URL and should be redirected
        #

        new_row['Redirect status'] = REDIRECT_STATUS_FOUND

        # 2. Determine if the canonical version of the URL is valid and the URL should be redirected to its canonical version
        canonical_match = df_valid_urls[df_valid_urls['URI'] == row['Canonical']]
        if not canonical_match.empty:
            # The URL in the 'Canonical' column is a valid URL and we should redirect to it
            new_row['Redirect URI'] = row['Canonical']

        # 3. Determine if the current redirect URL is a valid URL and the original URL should be redirected
        redirect_match = df_valid_urls[df_valid_urls['URI'] == row['Redirect URI']]
        if not redirect_match.empty:
            # Case 1: Current redirect URL is valid as the URL exists in df_valid_urls
            new_row['Redirect URI'] = row['Redirect URI']
        else:
            # Case 2: Check if there is a redirect mapping from an alias in `df_alias_redirects``
            alias_match = df_alias_redirects[df_alias_redirects['URI'] == row['Redirect URI']]
            if not alias_match.empty:
              new_row['Redirect URI'] = alias_match.iloc[0]['Redirect URI']  # First matching record's Redirect URL
            else:
                # No valid URL to redirect to found
                new_row['Redirect status'] = REDIRECT_STATUS_NOT_FOUND

    # Append the new row to the list
    merged_redirects_list.append(new_row)

# Convert the list of dictionaries to a DataFrame
df_merged_redirects = pd.DataFrame(merged_redirects_list)
df_merged_redirects

,URI,Canonical,Redirect URI,Redirect status,Count,Last
0,/,/,/,200,1944694,2024-01-07 12:32:07+00:00
1,/about,/about/,/about/,301,185537,2024-01-06 16:26:33+00:00
2,/atom.xml,/,/,301,178818,2024-01-07 09:54:37+00:00
3,/articles/2011/07/26/disable-recent-items,/articles/disable-recent-items/,/technology/disable-recent-items/,301,148956,2024-01-06 18:32:38+00:00
4,/articles/2012/07/10/time-machine-inherit-back...,/articles/time-machine-inherit-backup-using-tm...,/technology/time-machine-inherit-backup-using-...,301,104450,2024-01-07 12:21:01+00:00
...,...,...,...,...,...,...
7275,/articles/2009/,/articles/2009/,/articles/2009/,404,1,2013-12-16 19:45:29+00:00
7276,/articles/2009/11/,/articles/2009/11/,/articles/2009/11/,404,1,2013-12-16 19:45:29+00:00
7277,/articles/2011/03/17/cisco-vpn-10.6.0-3.html,/articles/cisco-vpn-10.6.0-3/,/technology/fix-cisco-vpn-disconnections-mac-o...,301,1,2013-12-12 16:06:00+00:00
7278,/articles/2009/01/10/time-machine-readonly.html,/articles/time-machine-readonly/,/technology/,301,1,2013-12-12 16:05:57+00:00


In [76]:
# Redirect some outdated URLs to the most relevant category or section to avoid 404 errors
default_rules = [
    (r'^/hints/macosx', r'^/hints/macosx(/server)?.*', r'/categories/macos/'),
    (r'^/hints.*', r'^/hints.*', r'/technology/'),
]

def default_url(row):
    redirect_url = row['Redirect URI']
    redirect_status = row['Redirect status']

    if redirect_status == REDIRECT_STATUS_NOT_FOUND:
        url = row['URI']

        # Apply transformation rules
        for pattern, search_pattern, replacement in default_rules:
            if re.match(pattern, url):
                redirect_url = re.sub(search_pattern, replacement, url)
                redirect_status = REDIRECT_STATUS_FOUND
                return pd.Series([redirect_url, redirect_status])

    return pd.Series([redirect_url, redirect_status])

# Apply the function and create 'df_redirects_raw'
df_merged_defaulted_redirects = df_merged_redirects.copy()
df_merged_defaulted_redirects[['Redirect URI', 'Redirect status']] = df_merged_defaulted_redirects.apply(default_url, axis=1, result_type='expand')
df_merged_defaulted_redirects

,URI,Canonical,Redirect URI,Redirect status,Count,Last
0,/,/,/,200,1944694,2024-01-07 12:32:07+00:00
1,/about,/about/,/about/,301,185537,2024-01-06 16:26:33+00:00
2,/atom.xml,/,/,301,178818,2024-01-07 09:54:37+00:00
3,/articles/2011/07/26/disable-recent-items,/articles/disable-recent-items/,/technology/disable-recent-items/,301,148956,2024-01-06 18:32:38+00:00
4,/articles/2012/07/10/time-machine-inherit-back...,/articles/time-machine-inherit-backup-using-tm...,/technology/time-machine-inherit-backup-using-...,301,104450,2024-01-07 12:21:01+00:00
...,...,...,...,...,...,...
7275,/articles/2009/,/articles/2009/,/articles/2009/,404,1,2013-12-16 19:45:29+00:00
7276,/articles/2009/11/,/articles/2009/11/,/articles/2009/11/,404,1,2013-12-16 19:45:29+00:00
7277,/articles/2011/03/17/cisco-vpn-10.6.0-3.html,/articles/cisco-vpn-10.6.0-3/,/technology/fix-cisco-vpn-disconnections-mac-o...,301,1,2013-12-12 16:06:00+00:00
7278,/articles/2009/01/10/time-machine-readonly.html,/articles/time-machine-readonly/,/technology/,301,1,2013-12-12 16:05:57+00:00


In [77]:
valid_redirects = df_merged_defaulted_redirects['Redirect status'] == REDIRECT_STATUS_FOUND
# Write the DataFrame to a CSV file
redirects_csv = './output/redirects.csv'
df_valid = df_merged_defaulted_redirects[valid_redirects]
df_valid.to_csv(redirects_csv, index=False)

df_output = df_valid.copy()
df_output = df_output[['URI', 'Redirect URI', 'Redirect status']]
df_output = df_output.rename(columns={'URI': 'path', 'Redirect URI': 'target', 'Redirect status': 'status'})

df_output = df_output.sort_values(by='path', key=lambda x: x.str.lower(), ascending=True)

hugo_data_redirects_json = '../../data/redirects.json'
df_output.to_json(hugo_data_redirects_json, orient='records', lines=False)
df_output
# df_invalid_redirects = df_output['status'] != REDIRECT_STATUS_FOUND
# df_output[df_invalid_redirects]

,path,target,status
1,/about,/about/,301
2151,/about.html,/about/,301
34,/archive,/,301
38,/archive/,/,301
1490,/archive/index.xml,/,301
...,...,...,...
448,/transformation/organizational-transformation-...,/transformation/how-objectives-key-results-fac...,301
1143,/transformation/why-okr-objectives-key-results...,/transformation/why-okr-objectives-key-results...,301
1281,/work,/transformation/,301
597,/work/,/transformation/,301


In [78]:
# Get all access to PHP files, ignoring invalid URLs

# Ignore invalid URLs
# invalid_urls_pattern = (r'^/+(?:[?_]+'
#     + r'|db|upload|include|mindmeld|mods|module|lib|package|page|ops|sql|shell|inc|feed|media|plus'
#     + r'|data|tmp|index|vendor|sites|stream|lock|scripts|40|export|root|license|style|java'
#     + r'|fw|image|img|layout|lang|mail'
#     + r'|s_|photo|wp[^a-z]|plug|pod|defa|bu|mt|core|theme'
#     + r'|xl|indox|pma|xx|tp|.well-known'
#     + r'|[^/]+\.php'
#     + r')'
#     + r'|laravel|lufix|beence|mxbb|mygallery|ossigeno|phorum|promocms|agsearch|kcfinder|n0way|explore'
#     + r'|assetmanager|assets|css|js|mybic|appserv|footer|nuclearbb|mser|creativecontact|betablock|downstat|docebo|dm-album|heimlicher|ispirit|logs|code|privilege'
#     + r'|wp-|adm|_fragment|wchat|phpstorm|server/php|phpinfo|phpunit|vtiger|fck|config|echo|system|template|elseif|password|tool'
#     + r'|xmlrpc|public_html|install|cgi|local-bin|register|test|phpmyadmin|myadmin|myaccount|atutor'
#     + r'|b2-tools|magmi|news|frame|mod_|cache|bridge|moteur|music|shop|script|process|skin|source'
#     + r'|class|alert|common|wapchat|ytb|recording|address|component|client|web|wsk|content|bins|public|upload|compteur|control|convert|product|runtime|dump|dialog|guestbook|gaestebuch|gastenboek|gb'
#     + r'|adaptcms|bpnews|user|gemini|snippetmaster|download|logging|contenido|modules|include|widget|squery|don3|templates|gallery|adodb|path|ajax|amazon|akarru|belegungsplan|bemarket|bigace'
#     + r'|(?:index|z|doc|wp|info|editor|dropdown|alfa|up|about|wso|moon|pipe|data|defaul1|team|setup|by|byp|cp|customize|default|md5|petx|apps|elfinder|timeclock|header|donation|links|search|viar|application|core'
#     + r'|edit-comments|fm|gecko|member|home|init|lufi|ws|log|plugins|repeater|books|action|util|article-raw|radio|elrekt|moduless|forum|mytag_js|reg|sky|protection|ee|auth|upl|google|marijuana|view|pollvote|popup_window|port|mysave|simple|browser|router|wiki|login|ups|olux|legion|sym|symmlink|error|term|tesla|lalala|leet|lydia|mar|/[a-z]|/[0-9]+)[^a-z]?\.php'
#     + r'|\bphp[^?]+'
# )
# # invalid_urls_pattern
# invalid_urls_mask = df['URI'].str.contains(invalid_urls_pattern, na=False, case=False)
# df_valid = df[~invalid_urls_mask]

In [542]:
# # Length of continuous sequence of URL-encoded characters
# encoded_seq_len = 1

# # Regular expression for URL escape sequences
# escape_seq_regex = r'(?:%[0-9A-Fa-f]{2}){' + f"{encoded_seq_len},{encoded_seq_len}" + r'}'

# # Function to find all escape sequences in a URL
# def find_escape_sequences(url):
#     return [seq.upper() for seq in re.findall(escape_seq_regex, url)]

# # Data structure to store the escape sequences, counts, and last access
# escape_data = defaultdict(lambda: {'count': 0, 'last_access': None})

# # Iterate over each row in the DataFrame
# for index, row in df.iterrows():
#     escape_sequences = find_escape_sequences(row['URI'])
#     for seq_upper in escape_sequences:
#         escape_data[seq_upper]['count'] += 1
#         if not escape_data[seq_upper]['last_access'] or row['Last'] > escape_data[seq_upper]['last_access']:
#             escape_data[seq_upper]['last_access'] = row['Last']

# # Convert the escape data to a DataFrame and include the decoded character
# df_escape_sequences = pd.DataFrame([
#     {
#         'Escape Sequence': seq,
#         'Decoded Characters': unquote(seq),
#         'Total Access Count': data['count'],
#         'Last': data['last_access']
#     }
#     for seq, data in escape_data.items()
# ])

# df_escape_sequences.sort_values(by='Total Access Count', ascending=False, inplace=True)
# covered_lines = set()
# selected_sequences = set()

# # Iterate over the sorted escape sequences DataFrame
# for index, row in df_escape_sequences.iterrows():
#     seq = row['Escape Sequence']

#     # Find all lines in the original DataFrame that contain this sequence
#     matching_lines = {idx for idx, url in enumerate(df['URI']) if seq in url}

#     # Check if this sequence covers new lines
#     new_lines = matching_lines - covered_lines
#     if new_lines:
#         # This sequence covers new lines, add it to the selected sequences
#         selected_sequences.add(seq)
#         covered_lines.update(new_lines)

#     # Check if all lines are covered
#     if len(covered_lines) == len(df):
#         break

# # selected_sequences now contains the minimal set of sequences
# print(selected_sequences)

In [11]:
# Get the Top 100 Accessed URLs with status code 301, ordered by last access

# Filter for status code 301
df_301 = df[df['Status Code'] == 301]

# Group by URL and sum 'Count', then get the top 100
top_100 = df_301.groupby('URI')['Count'].sum().nlargest(100).reset_index(name='Total Access Count')

In [9]:
# Merge to get the last access times
top_100_with_dates = top_100.merge(df_301[['URI', 'Last']], on='URI', how='left')

# Drop duplicates to keep only the most recent access for each URL
top_100_with_dates = top_100_with_dates.drop_duplicates(subset=['URI'])

import urllib

# Not necessary: Encoding URLs to handle special characters
# top_100_with_dates['Clean URL'] = top_100_with_dates['URI'].apply(lambda x: urllib.parse.quote_plus(x))

# Sort by 'Last' in descending order
top_100_with_dates_sorted = top_100_with_dates.sort_values(by='Last', ascending=False)

In [10]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# Enable the output of bokeh plots in Jupyter Notebook
output_notebook()

source_df = top_100_with_dates_sorted

# Create a ColumnDataSource from the top 10 data
source = ColumnDataSource(data=source_df)

# Create the figure
p = figure(y_range=source_df['URI'], height=1600, width=800, title="Top 10 Frequently Accessed URLs with Status Code 301", tools="pan,box_zoom,reset")

# Add horizontal bar plot
p.hbar(y='URI', right='Total Access Count', source=source, height=0.4)

# Add labels
p.xaxis.axis_label = 'Number of Accesses'
p.yaxis.axis_label = 'URLs'

# Show the plot
show(p)

Loading BokehJS ...

In [ ]:
# Get all access to PHP files, ignoring invalid URLs

# Ignore invalid URLs
# invalid_urls_pattern = (r'^/+(?:[?_]+'
#     + r'|db|upload|include|mindmeld|mods|module|lib|package|page|ops|sql|shell|inc|feed|media|plus'
#     + r'|data|tmp|index|vendor|sites|stream|lock|scripts|40|export|root|license|style|java'
#     + r'|fw|image|img|layout|lang|mail'
#     + r'|s_|photo|wp[^a-z]|plug|pod|defa|bu|mt|core|theme'
#     + r'|xl|indox|pma|xx|tp|.well-known'
#     + r'|[^/]+\.php'
#     + r')'
#     + r'|laravel|lufix|beence|mxbb|mygallery|ossigeno|phorum|promocms|agsearch|kcfinder|n0way|explore'
#     + r'|assetmanager|assets|css|js|mybic|appserv|footer|nuclearbb|mser|creativecontact|betablock|downstat|docebo|dm-album|heimlicher|ispirit|logs|code|privilege'
#     + r'|wp-|adm|_fragment|wchat|phpstorm|server/php|phpinfo|phpunit|vtiger|fck|config|echo|system|template|elseif|password|tool'
#     + r'|xmlrpc|public_html|install|cgi|local-bin|register|test|phpmyadmin|myadmin|myaccount|atutor'
#     + r'|b2-tools|magmi|news|frame|mod_|cache|bridge|moteur|music|shop|script|process|skin|source'
#     + r'|class|alert|common|wapchat|ytb|recording|address|component|client|web|wsk|content|bins|public|upload|compteur|control|convert|product|runtime|dump|dialog|guestbook|gaestebuch|gastenboek|gb'
#     + r'|adaptcms|bpnews|user|gemini|snippetmaster|download|logging|contenido|modules|include|widget|squery|don3|templates|gallery|adodb|path|ajax|amazon|akarru|belegungsplan|bemarket|bigace'
#     + r'|(?:index|z|doc|wp|info|editor|dropdown|alfa|up|about|wso|moon|pipe|data|defaul1|team|setup|by|byp|cp|customize|default|md5|petx|apps|elfinder|timeclock|header|donation|links|search|viar|application|core'
#     + r'|edit-comments|fm|gecko|member|home|init|lufi|ws|log|plugins|repeater|books|action|util|article-raw|radio|elrekt|moduless|forum|mytag_js|reg|sky|protection|ee|auth|upl|google|marijuana|view|pollvote|popup_window|port|mysave|simple|browser|router|wiki|login|ups|olux|legion|sym|symmlink|error|term|tesla|lalala|leet|lydia|mar|/[a-z]|/[0-9]+)[^a-z]?\.php'
#     + r'|\bphp[^?]+'
# )
# # invalid_urls_pattern
# invalid_urls_mask = df['URI'].str.contains(invalid_urls_pattern, na=False, case=False)
# df_valid = df[~invalid_urls_mask]